In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv()

True

In [5]:
from langchain_teddynote import logging
logging.langsmith("portfolio-interview-qa")

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. PDF 로딩 및 분할
loader = PyMuPDFLoader("data.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# 2. 프로젝트/기술스택 요약
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert IT recruiter. Summarize the main projects and technical stacks from the following portfolio text. Respond in bullet points. Only return the summary, no explanations."),
    ("human", "{input}")
])
summary_chain = summary_prompt | llm | StrOutputParser()
summaries = [summary_chain.invoke({"input": chunk.page_content}) for chunk in chunks]
full_summary = "\n".join(summaries)

# 3. 고퀄리티 질문 생성 (문제 해결 질문 포함, 한국어)
question_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a senior software engineer and technical interviewer. "
     "Based on the following portfolio summary, generate exactly 5 high-quality, in-depth interview questions in Korean. "
     "3 questions must be based on the candidate's projects/experience, and 2 questions must be based on their technical stack. "
     "Among the project-based questions, at least one must ask in detail about a specific problem or challenge the candidate faced during a project and how they solved or overcame it. "
     "Questions should require deep reasoning, real-world application, and critical thinking. "
     "Be creative and avoid generic questions. "
     "Label each question as [Portfolio] or [Tech Stack]. "
     "Return only the questions as a numbered list. "
     "All questions must be written in Korean."),
    ("human", "{summary}")
])
question_chain = question_prompt | llm | StrOutputParser()
questions = question_chain.invoke({"summary": full_summary})

# 4. 질문을 txt 파일로 저장
with open("interview_questions.txt", "w", encoding="utf-8") as f:
    f.write(questions)

print("면접 질문이 interview_questions.txt 파일에 저장되었습니다.")

LangSmith 추적을 시작합니다.
[프로젝트명]
portfolio-interview-qa
면접 질문이 interview_questions.txt 파일에 저장되었습니다.


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Q&A txt 파일 읽기
with open("interview_qa.txt", "r", encoding="utf-8") as f:
    qa_text = f.read()

criteria_feedback = """
아래의 각 평가 항목별로 지원자의 답변에 대해 매우 엄격하고 구체적으로 피드백을 작성하세요. 점수는 절대 포함하지 마세요.  
각 항목별로 한 문단 이상의 상세 피드백을 작성하세요.

[기술 질문 평가 항목]
- 핵심 기술 개념의 이해도
- 비교 및 분석 능력
- 성능 최적화 고려
- 최신 트렌드 반영
- 기술 용어의 정확성과 일관성

[포트폴리오 질문 평가 항목]
- 프로젝트 기반 설명력
- 문제 해결 과정의 명확성
- 결과 도출 및 개선 노력
- 실무 적용 가능성
- 협업 및 기여도 표현

피드백은 반드시 한국어로 작성하세요.
"""

feedback_prompt = ChatPromptTemplate.from_messages([
    ("system", f"당신은 시니어 소프트웨어 엔지니어이자 기술 면접관입니다. 아래의 평가 기준에 따라 지원자의 답변을 평가하세요.\n{criteria_feedback}\n"),
    ("human", "{qa_text}")
])
feedback_chain = feedback_prompt | llm | StrOutputParser()
feedback_only = feedback_chain.invoke({"qa_text": qa_text})

with open("interview_feedback.txt", "w", encoding="utf-8") as f:
    f.write(feedback_only)
print("상세 피드백이 interview_feedback.txt 파일에 저장되었습니다.")

상세 피드백이 interview_feedback.txt 파일에 저장되었습니다.


In [7]:
# 피드백 파일 읽기
with open("interview_feedback.txt", "r", encoding="utf-8") as f:
    feedback_text = f.read()

criteria_score = """
아래의 각 평가 항목별로 0~100점 만점으로 매우 엄격하게 점수를 매기세요.
아래 형식만 출력하세요(불필요한 설명, 번호, 공백 없이):

핵심 기술 개념의 이해도: X/100
비교 및 분석 능력: X/100
성능 최적화 고려: X/100
최신 트렌드 반영: X/100
기술 용어의 정확성과 일관성: X/100
프로젝트 기반 설명력: X/100
문제 해결 과정의 명확성: X/100
결과 도출 및 개선 노력: X/100
실무 적용 가능성: X/100
협업 및 기여도 표현: X/100

반드시 위 10개 항목만, '항목명: X/100' 형식으로 한 줄씩 출력하세요.  
점수만 출력하고, 다른 설명은 절대 포함하지 마세요.
"""

score_prompt = ChatPromptTemplate.from_messages([
    ("system", f"당신은 시니어 소프트웨어 엔지니어이자 기술 면접관입니다. 아래의 피드백을 읽고 각 항목별 점수를 산출하세요.\n{criteria_score}\n"),
    ("human", "{feedback_text}")
])
score_chain = score_prompt | llm | StrOutputParser()
scores_only = score_chain.invoke({"feedback_text": feedback_text})

with open("interview_scores.txt", "w", encoding="utf-8") as f:
    f.write(scores_only)
print("점수만 interview_scores.txt 파일에 저장되었습니다.")

점수만 interview_scores.txt 파일에 저장되었습니다.
